In [12]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF

In [13]:
#导入数据
user_movie_train = pd.read_table('/data/data/UserMovie_train.txt')
data = user_movie_train

In [14]:
data.head()

,uid,mid,timeStamp,star
0,1722994,1306505,2007-08-22,5.0
1,1405477,10574468,2013-04-24,4.0
2,15849871,4910186,2011-12-20,5.0
3,1068524,1304643,2006-02-07,3.0
4,1307041,1851857,2008-09-22,4.0


In [17]:
testuser = pd.read_csv('UserMovie_test2.txt',names=['uid'])

In [18]:
testuser.head()

,uid
0,1000001
1,1000023
2,1000030
3,1000043
4,1000045


In [27]:
cross_user = pd.merge(testuser,data,on=['uid'],how='left')

In [28]:
cross_user.head()

,uid,mid,timeStamp,star
0,1000001,1301578,2005-06-12,4.0
1,1000001,3196716,2011-07-01,4.0
2,1000001,1293590,2005-06-12,4.0
3,1000001,3066739,2010-05-10,3.0
4,1000001,1766827,2007-11-26,4.0


In [31]:
len(cross_user)

5249324

In [30]:
test = cross_user.dropna()

In [32]:
len(test)

5248815

In [20]:
cross_user.head()

,uid,mid,timeStamp,star
0,1722994,1306505,2007-08-22,5.0
1,1722994,1439831,2007-08-24,5.0
2,1722994,1851906,2007-08-23,4.0
3,1722994,1295644,2008-11-02,5.0
4,1722994,1465037,2007-08-17,5.0


In [39]:
mid = cross_user['mid'].unique()
uid = cross_user['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
cross_user['mid'] = cross_user['mid'].map(lambda x:collect_mid_dencoder[x])
cross_user['uid'] = cross_user['uid'].map(lambda x:collect_uid_dencoder[x])

In [40]:
user_matrix_cross = np.zeros((len(uid),len(mid)),dtype='float16')
item_matrix_cross = np.zeros((len(mid),len(uid)),dtype='float16')
to_make = np.array(cross_user[['uid','mid','star']].values,dtype=np.uint32)
for i in range(len(to_make)):
    if i%1000000 == 0:
        print(i)
    user_matrix_cross[to_make[i][0],to_make[i][1]] = to_make[i][2]
    item_matrix_cross[to_make[i][1],to_make[i][0]] = to_make[i][2]

0
1000000
2000000
3000000
4000000
5000000


In [4]:
# uid 48613  mid 72541
mid = data['mid'].unique()
uid = data['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data['mid'] = data['mid'].map(lambda x:collect_mid_dencoder[x])
data['uid'] = data['uid'].map(lambda x:collect_uid_dencoder[x])

In [5]:
user_matrix = np.zeros((len(uid),len(mid)),dtype='float16')
item_matrix = np.zeros((len(mid),len(uid)),dtype='float16')
data.head(5)

,uid,mid,timeStamp,star
0,0,0,2007-08-22,5.0
1,1,1,2013-04-24,4.0
2,2,2,2011-12-20,5.0
3,3,3,2006-02-07,3.0
4,4,4,2008-09-22,4.0


In [6]:
to_make = np.array(data[['uid','mid','star']].values,dtype=np.uint32)
for i in range(len(to_make)):
    if i%1000000 == 0:
        print(i)
    user_matrix[to_make[i][0],to_make[i][1]] = to_make[i][2]
    item_matrix[to_make[i][1],to_make[i][0]] = to_make[i][2]

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000


In [41]:
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    
def calc_sim(vec,user_matrix):
    ans = []
    for i in range(len(user_matrix)):
#         if i % 10000 ==0:
#             print(i)
        #ans.append(np.linalg.norm(vec-user_matrix[i]))
        ans.append(cos_sim(vec,user_matrix[i]))
    return ans

In [45]:
user_matrix_cross

array([[4., 4., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 5., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 4.]], dtype=float16)

In [ ]:
import random
now = 0
final_acc = []
#从usr_matrix中取出一位用户
rec_list=[]
for new_vec in user_matrix_cross[100]:
    #ans存了
    ans = calc_sim(new_vec,user_matrix_cross)
    to_recommand = DF(ans).reset_index().rename(columns={0:'sim'})
    k = 3
    #找出最相似的三个用户
    topk = list(to_recommand.sort_values(by=['sim'],ascending=False).head(k)['index'].values)
#     print(topk)
#     no_to_use = []
#     for i in range(len(new_vec)):
#         if random.randint(0,100) % 10 == 0:
#             if new_vec[i]!=0:
#                 no_to_use.append(i)
    
    # 推荐没给他看过的
    watch_set = set()
    #求三个用户的并集
    for i in topk:
        watch_set = watch_set|set(cross_user[cross_user['uid']==i]['mid'].unique())
    watch_set = set(watch_set)
    #print('The New One No Watch Movie Length: ',len(watch_set))
    topN = []
    for i in list(watch_set):
        temp = 0
        for j in topk:
            temp += to_recommand[to_recommand['index']==j]['sim'].values*user_matrix[j][i]
        topN.append(temp)
    topN = DF(topN).rename(columns={0:'score'})
    topN['mid'] = watch_set
    need_to_show = 50
    show_video = topN.sort_values(by=['score'],ascending=False).head(need_to_show)['mid'].values
    true_video = [collect_mid_encoder[i] for i in show_video]
    #print(true_video)
    rec_list.append(true_video)
#     print(show_video)
    print(true_video)
#     print(topN.sort_values(by=['score'],ascending=False).head(need_to_show))
#     acc = data[(data['mid'].isin(show_video) & (data['uid']==now))].shape[0]/need_to_show
#     print('True: ',acc)
#     final_acc.append(acc) 
#     now += 1